In [20]:
IRdisplay::display_html(file='code_hiding.html')

Toggle ON/OFF 
code cells.

In [1]:
# load packages and define constants
library(data.table) # see https://cran.r-project.org/web/packages/data.table/vignettes/datatable-intro.html for reference
library(ggplot2)
source("R_functions.r")

# folder/file-specific constants
PILOT_NUMBER <- 6
DATA_FOLDER <- "~/programing/data/psychophys/"
FIRA_TAG <- "FIRA"
FRAMES_TAG <- "framesInfo"
DOTS_TAG <- "dotsPositions"

# key-specific constants 
TRIALS <- "trials"
FRAMES <- "frames"
DOTS <- "dots"

In [2]:
# load csv files into data.tables
tb <- list(
    loadPilotCSV(PILOT_NUMBER, DATA_FOLDER, FIRA_TAG),
    loadPilotCSV(PILOT_NUMBER, DATA_FOLDER, FRAMES_TAG),
    loadPilotCSV(PILOT_NUMBER, DATA_FOLDER, DOTS_TAG))
names(tb) <- c(TRIALS, FRAMES, DOTS)

# Notebook's goals

Make sure the workflow from running the task to analyzing the data works.


## Workflow description
1. Task is run with repo [SingleCP_DotsReversal_Task](https://github.com/TheGoldLab/SingleCP_DotsReversal_Task/) (appropriate branch must be chosen).
  1. A `.mat` data file is outputted
  2. I usually manually rename this file `pilot_#.mat` and upload it to PennBox (Data/Psychophysics/Radillo_SingleCP_DotsReversal/)
2. Data is analyzed with repo [SingleCP_DotsReversal_DataAnalaysis](https://github.com/aernesto/SingleCP_DotsReversal_DataAnalysis)(again, with the appropriate branch).
  1. The first step is to convert data from the .mat file into CSV format
    - `pilot#_framesInfo.csv` produced with [this script](https://github.com/aernesto/SingleCP_DotsReversal_DataAnalysis/blob/25d37b8a9cb2fb768359dd30be4452aed60b9c62/MATLAB_scripts/explore_data_file.m) as of 01/28/2019. [Fields description](https://github.com/aernesto/SingleCP_DotsReversal_DataAnalysis/wiki/Fields-description-of-*framesInfo.csv-file).
    - `pilot#_FIRA.csv` produced with [this script](https://github.com/aernesto/SingleCP_DotsReversal_DataAnalysis/blob/25d37b8a9cb2fb768359dd30be4452aed60b9c62/MATLAB_scripts/explore_data_file.m) as of 01/28/2019. [Fields description](https://github.com/aernesto/SingleCP_DotsReversal_DataAnalysis/wiki/Fields-Description-of-*FIRA.csv-files).
    - `pilot#_dotsPositions.csv` produced with same script as above, as of 01/28/2019. [Fields description](https://github.com/aernesto/SingleCP_DotsReversal_DataAnalysis/wiki/Fields-descriptions-for-*dotsPositions.csv-file).

**TO-DO**
- Write detailed explanation of each column in each `.csv` file (i.e. update the [Wiki](https://github.com/aernesto/SingleCP_DotsReversal_DataAnalysis/wiki))
- Make sure data from the three `.csv` files is consistent

## Specific questions
1. How many frames are skipped on each trial, and where in the trial do they occur?
2. Does this number match the offset in viewing duration per trial?
3. Compute reverse kernels with the number of coherent dots as a proxy for motion energy

# Exploring the data
## The `*FIRA.csv` file (trials level)

In [3]:
str(tb[[TRIALS]])
unique(tb[[TRIALS]][,viewingDuration])

Classes ‘data.table’ and 'data.frame':	18 obs. of  25 variables:
 $ taskID         : int  2 2 2 2 2 2 2 2 2 2 ...
 $ trialIndex     : int  6 9 12 14 1 15 4 17 11 10 ...
 $ trialStart     : num  805061 805066 805071 805076 805081 ...
 $ trialEnd       : num  805066 805071 805076 805081 805085 ...
 $ RT             : num  1.222 1.223 1.1 0.703 1.228 ...
 $ choice         : int  1 0 0 0 0 1 0 1 1 0 ...
 $ correct        : int  1 0 0 1 1 0 1 1 1 0 ...
 $ initDirection  : int  0 0 0 0 180 180 180 180 0 0 ...
 $ endDirection   : int  0 0 0 180 180 180 180 0 0 0 ...
 $ presenceCP     : int  0 0 0 1 0 0 0 1 0 0 ...
 $ coherence      : num  25.6 12.8 25.6 6.4 6.4 12.8 12.8 25.6 25.6 12.8 ...
 $ viewingDuration: num  0.1 0.2 0.2 0.3 0.1 0.3 0.1 0.3 0.2 0.2 ...
 $ probCP         : num  0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 ...
 $ timeCP         : num  0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 ...
 $ randSeedBase   : int  7597 9958 1108 5553 2607 9306 2457 859 8670 7918 ...
 $ fixationOn     : num

[1] 0.1 0.2 0.3

## The `*framesInfo.csv` file (frames level)

In [4]:
str(tb[[FRAMES]])
unique(tb[[FRAMES]]$trialIndex)

Classes ‘data.table’ and 'data.frame':	314 obs. of  6 variables:
 $ frameTotCount: int  3 4 5 6 7 8 9 10 11 12 ...
 $ onsetTime    : num  805062 805063 805064 805064 805064 ...
 $ onsetFrame   : int  215 290 345 351 352 353 508 600 624 625 ...
 $ swapTime     : num  805062 805063 805064 805064 805064 ...
 $ isTight      : int  0 0 0 0 1 1 0 0 0 1 ...
 $ trialIndex   : int  6 6 6 6 6 6 9 9 9 9 ...
 - attr(*, ".internal.selfref")=<externalptr> 


[1]  6  9 12 14  1 15  4 17 11 10  3  5 13  8 18  2  7 16

In [5]:
tb[[FRAMES]][order(onsetFrame),.(minOnset=min(onsetFrame), maxOnset=max(onsetFrame), numUniqueOnset=.N), by=trialIndex]

trialIndex,minOnset,maxOnset,numUniqueOnset
6,215,353,6
9,508,637,17
12,787,911,16
14,1068,1219,24
1,1342,1451,11
15,1601,1748,24
4,1902,2082,11
17,2232,2420,24
11,2567,2709,18
10,2849,3005,17


In [19]:
tb[[FRAMES]][trialIndex==16,sum(isTight) ]

[1] 22

## The `*dotsPositions.csv` file (dots level)

In [18]:
tb[[DOTS]][trialCount == 18,.N,by=frameIdx]

frameIdx,N
1,182
2,182
3,182
4,182
5,182
6,182
7,182
8,182
9,182
10,182


In [8]:
str(tb[[DOTS]])
#tb[[DOTS]][,.(.N),by=.(frameIdx,trialCount)]

Classes ‘data.table’ and 'data.frame':	43862 obs. of  6 variables:
 $ xpos      : num  0.7459 0.5048 0.8971 0.3443 0.0796 ...
 $ ypos      : num  0.2885 0.8416 0.5606 0.5963 0.0589 ...
 $ isActive  : int  1 0 0 1 0 0 1 0 0 1 ...
 $ isCoherent: int  1 0 0 1 0 0 1 0 0 1 ...
 $ frameIdx  : int  1 1 1 1 1 1 1 1 1 1 ...
 $ trialCount: int  1 1 1 1 1 1 1 1 1 1 ...
 - attr(*, ".internal.selfref")=<externalptr> 
 - attr(*, "index")= int 
  ..- attr(*, "__trialCount")= int 


# Checking that the three datasets are consistent
As we can see, the `frameIdx` field is common to `tb[[FRAMES]]` and `tb[[DOTS]]`, and the `trialIdx` is common to `tb[[DOTS]]` and `tb[[TRIALS]]`. Let's check whether these common fields match in terms of their unique values in each dataset.
## Exploring match between frames and dots levels

In [9]:
length(unique(tb[[FRAMES]][,frameTotCount]))

[1] 314

In [10]:
unique(tb[[DOTS]][,frameIdx])
length(unique(tb[[DOTS]][,frameIdx]))

[1]  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21

[1] 21

## Exploring match between `DOTS` and `TRIALS` levels

In [11]:
unique(tb[[DOTS]][,trialCount])

[1]  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18

In [12]:
unique(tb[[TRIALS]][,trialIndex])
length(unique(tb[[TRIALS]][,trialIndex]))
min(unique(tb[[TRIALS]][,trialIndex]))
max(unique(tb[[TRIALS]][,trialIndex]))

[1]  6  9 12 14  1 15  4 17 11 10  3  5 13  8 18  2  7 16

[1] 18

[1] 1

[1] 18

## Exploring match between `FRAMES` and `TRIALS` levels

In [13]:
setkey(tb[[FRAMES]], trialIndex)
setkey(tb[[TRIALS]], trialIndex)

# Full OUTER JOIN (see https://rstudio-pubs-static.s3.amazonaws.com/52230_5ae0d25125b544caab32f75f0360e775.html)
frameCount <- merge(
    tb[[FRAMES]][order(trialIndex),.(numFramesInFRAMES=.N),by=trialIndex],
    tb[[TRIALS]][order(trialIndex),.(numFramesInTRIALS=numFrames),by=trialIndex],
    all=TRUE)
frameCount[,.(trialIndex, numFramesInFRAMES, numFramesInTRIALS, match=numFramesInFRAMES == numFramesInTRIALS)]

trialIndex,numFramesInFRAMES,numFramesInTRIALS,match
1,11,11,TRUE
2,11,11,TRUE
3,11,11,TRUE
4,11,11,TRUE
5,12,12,TRUE
6,6,6,TRUE
7,18,18,TRUE
8,19,19,TRUE
9,17,17,TRUE
10,17,17,TRUE


Any non-match in the right-most column above signals an *issue*.